In [1]:
import urllib
import requests
import itertools
from copy import deepcopy as mcopy
from music21 import *
import threading
import time

from ksxml import permute, \
    nth_permutation_indices, \
    send_m21_object, \
    write_midi_file, \
    send_transport_command, \
    stream_generator_from_chord_arrays, \
    request_svg_creation

In [2]:
# for generating SVGs asyncronously
def stop_creating_images():
    for thread in threading.enumerate():
        if thread.isAlive() and thread.name == 'ks-image-gen':
            thread.cancel()

def create_images(streams, name_prefix, offset, current_index = 0):
    if current_index >= len(streams): return
    name = name_prefix + str(current_index + offset)
    res = request_svg_creation(streams[current_index], name)

    t = threading.Timer(0.5, create_images,
                        [streams, name_prefix, offset, current_index + 1])
    t.name = 'ks-image-gen'
    t.daemon = True
    t.start()

In [3]:
# source material
n1 = ['C3', 'D3', 'G3', 'A3']
n2 = ['C3', 'e3', 'G3', 'b3'] 
n3 = ['D3', 'E3', 'A3', 'B3']

chords_with_names = [n1, n1, n2, n2, n3, n3]

In [4]:
start_index = 0
count = 1000

stream_gen = stream_generator_from_chord_arrays(chords_with_names, start_index, count)
streams = [s for s in stream_gen]
full_stream = stream.Stream()
for s, i in zip(streams, range(start_index, count)):
    s.insert(clef.BassClef())
    full_stream.append(s)
    
# write_midi_file(full_stream.flat, '/session/default.mid')
# send_transport_command('a', 'reload')

In [13]:
# Create images from the 
create_images(streams, 'htv00-', start_index)

In [6]:
a = request_svg_creation(streams[90], 'htv00-')
a.json()

{'path': '/score-img/htv00-.svg'}

In [10]:
stop_creating_images()

In [8]:
# Push score to browser client from a m21 stream object
send_m21_object('a', streams[100])

<Response [200]>

In [11]:
# Write a score file
write_midi_file(streams[0], '/session/default.mid')

In [15]:
# Transport control via browser
send_transport_command('a', 'imagePrefix', {'value':'htv00-'})
send_transport_command('a', 'offset', {'value':0})
send_transport_command('a', 'play')

<Response [200]>

In [16]:
send_transport_command('a', 'stop')

<Response [200]>

In [9]:
send_transport_command('a', 'reload')

<Response [200]>

In [6]:
request_svg_creation(streams[9], 'test2')

<Response [200]>